In [1]:
import numpy as np, matplotlib.pyplot as plt, matplotlib as mpl
import pcmdpy_gpu as ppy
import pandas as pd
from tqdm import tqdm
from importlib import util
from seaborn import color_palette
%matplotlib inline

In [2]:
plt.style.use('cook')

In [3]:
from setup_files.mocks_paper1.mock_models import models, run_names, results

/n/home01/bcook/pCMDs/pixcmd/pcmdpy/pcmdpy/plotting/results.py:83: RuntimeWarning: invalid value encountered in sqrt
  logzerrs.append(np.sqrt(logzvar))
/n/home01/bcook/pCMDs/pixcmd/pcmdpy/pcmdpy/plotting/results.py:83: RuntimeWarning: invalid value encountered in sqrt
  logzerrs.append(np.sqrt(logzvar))
/n/home01/bcook/.conda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/n/home01/bcook/.local/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


# Setup

In [4]:
known_keys = ['use_gpu', 'verbose', 'dynamic', 'Nim', 'filters', 'iso_model', 'gal_model', 'like_mode', 'bins', 'downsample', 'mag_system', 'lum_cut', 'fixed_seed', 'sky_noise', 'shot_noise', 'prior', 'data_is_mock', 'data_pcmd', 'sampler_seed']

In [5]:
f_mock = ppy.instrument.default_m31_filters()

In [6]:
iso_model = ppy.isochrones.Isochrone_Model(f_mock, mag_system='vega')

In [7]:
driv = ppy.driver.Driver(iso_model, gpu=True, max_Nim=2048)

Please wait while the GPU states are initialized

In [8]:
def get_max_logl(key, n_samples=100, verbose=True):
    config_mod = 'setup_files/mocks_paper1'
    config_file = f'setup_files/mocks_paper1/{key}.py'
    spec = util.spec_from_file_location(config_mod, config_file)
    config = util.module_from_spec(spec)
    spec.loader.exec_module(config)
    params = config.params
    
    for k in params.keys():
        assert (k in known_keys), f"Key unknown: {k}"
    assert params['data_is_mock'], "Have not implemented data runs yet"
    
    driv.filters = params['filters']
    driv.iso_model = params['iso_model']
    driv.initialize_data(params['data_pcmd'], bins=params['bins'])
    best_params = results[key].best_params
    Nim = params['Nim']
    gal_model = params['gal_model']
    prior = params.get('prior', gal_model.get_flat_prior())
    
    logl_kwargs = {'like_mode': params['like_mode'],
                  'downsample': params['downsample'],
                  'mag_system': params['mag_system'],
                  'sky_noise': params['sky_noise'],
                  'shot_noise': params['shot_noise']}
    
    logls = [ppy.fit_model.lnlike(best_params, driv, Nim, prior.lnprior, gal_model, fixed_seed=False,
                                 **logl_kwargs) for _ in tqdm(range(n_samples), disable=(not verbose))]
    return np.median(logls)

In [15]:
def add_max_logl(key, n_samples=100, verbose=True):
    filename = f'results/paper1_{key}.csv'
    with open(filename, 'r') as f:
        text = f.read()
    if 'max_logl' in text:
        return float(f.partition('\n')[0].partition(': ')[-1])
    max_logl = get_max_logl(key, n_samples=n_samples, verbose=verbose)
    with open(filename, 'w') as f:
        f.write(f'# max_logl : {max_logl:.5e}\n')
        f.write(text)
    return max_logl

In [16]:
still_running = ['mock_8', 'mock_11', 'mock_43', 'mock_44', 'mock_45', 'mock_46', 'mock_47', 'mock_48', 'mock_49', 'mock_50', 'mock_51', 'mock_52', 'mock_53', 'mock_54',
            'mismatch_2', 'mismatch_5', 'mismatch_7', 'mismatch_8', 'mismatch_9']

In [17]:
max_logls = {}

In [18]:
max_logls = {'mock_1': -1048.0260482946496,
'mock_10': -929.7155038783135,
'mock_12': -1080.2963390595726,
'mock_13': -1086.925121590621,
'mock_14': -128.17944259664512,
'mock_15': -1437.1511937166442,
'mock_16': -2786.504479611057,
'mock_17': -11987.954270420698,
'mock_2': -82.71810734160911,
'mock_3': -940.9430895202383,
'mock_4': -416.2755513309453,
'mock_5': -1005.7536567925123,
'mock_6': -1021.558474398114,
'mock_7': -1010.3042785130085,
'mock_9': -1046.2162103566493,
'mock_18': -998.8989946671368,
'mock_19': -978.7855810806744,
'mock_20': -1020.1670255424187,
'mock_21': -990.2648116614484,
'mock_22': -1009.8070719964292,
'mock_23': -979.6290968971481,
'mock_24': -994.4396764285761,
'mock_25': -403.7302945670733,
'mock_26': -167.6606627593378,
'mock_27': -211.94085213446903,
'mock_28': -83.52576382051102,
'mock_29': -430.37949465113263,
'mock_30': -2938.4770232065757,
'mock_31': -400.99325799646,
'mock_32': -2933.283409560953,
'mock_33': -1005.4891672066396,
'mock_34': -1060.7678521808084,
'mock_35': -982.4428628647288,
'mock_36': -1044.525215398622,
'mock_37': -962.7595968491489,
'mock_38': -1028.4894456179945,
'mock_39': -1005.020253163382}

In [19]:
max_logls

{'mock_1': -1048.0260482946496,
 'mock_10': -929.7155038783135,
 'mock_12': -1080.2963390595726,
 'mock_13': -1086.925121590621,
 'mock_14': -128.17944259664512,
 'mock_15': -1437.1511937166442,
 'mock_16': -2786.504479611057,
 'mock_17': -11987.954270420698,
 'mock_18': -998.8989946671368,
 'mock_19': -978.7855810806744,
 'mock_2': -82.71810734160911,
 'mock_20': -1020.1670255424187,
 'mock_21': -990.2648116614484,
 'mock_22': -1009.8070719964292,
 'mock_23': -979.6290968971481,
 'mock_24': -994.4396764285761,
 'mock_25': -403.7302945670733,
 'mock_26': -167.6606627593378,
 'mock_27': -211.94085213446903,
 'mock_28': -83.52576382051102,
 'mock_29': -430.37949465113263,
 'mock_3': -940.9430895202383,
 'mock_30': -2938.4770232065757,
 'mock_31': -400.99325799646,
 'mock_32': -2933.283409560953,
 'mock_33': -1005.4891672066396,
 'mock_34': -1060.7678521808084,
 'mock_35': -982.4428628647288,
 'mock_36': -1044.525215398622,
 'mock_37': -962.7595968491489,
 'mock_38': -1028.4894456179945,


In [20]:
keys = set(results.keys())
keys -= set(still_running)
keys -= set(max_logls.keys())
for k in tqdm(list(keys)):
    if k in still_running:
        continue
    if k in max_logls:
        continue
    max_logl = add_max_logl(k, n_samples=100, verbose=False)
    max_logls[k] = max_logl
    print(k, max_logl)

 12%|█▎        | 1/8 [02:52<20:10, 172.97s/it]

mismatch_1 -961.535953064104


 25%|██▌       | 2/8 [04:34<15:09, 151.60s/it]

mismatch_4 -1050.8625584933295


 38%|███▊      | 3/8 [07:47<13:40, 164.04s/it]

...

mock_42 -569.881050639334


 88%|████████▊ | 7/8 [20:48<03:37, 217.55s/it]

mock_41 -1596.3366663487168


100%|██████████| 8/8 [22:14<00:00, 178.01s/it]

mismatch_6 -5441.489591979748


In [23]:
df_logl = pd.DataFrame.from_dict(max_logls, 'index', columns=['max_logl'])
df_logl.head()

,max_logl
mock_1,-1048.026048
mock_10,-929.715504
mock_12,-1080.296339
mock_13,-1086.925122
mock_14,-128.179443


In [24]:
df_logl.to_csv('setup_files/mocks_paper1/max_logls_post.csv', float_format='%.3f', index_label='run_name')